In [ ]:
"""
Installing the dependencies.
"""

!pip install pandas
!pip install scikit-learn
!pip install opencv-python
!pip install numpy
!pip install tensorflow
!pip install ultralytics

In [11]:

"""
Read the dataset and transform the labels to 0 or 1.
"""
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("image_labels.csv")
df['label'] = LabelEncoder().fit_transform(df['label'])
df["label"] = df["label"].replace({0: 1, 1: 0})

In [12]:
"""
Split the dataset into train, validation and test sets.
"""

X_train, X_test, y_train, y_test = train_test_split(df['filename'], df['label'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [13]:
"""
Read, resize and load images.
"""

import cv2
import numpy as np

def load_images(image_paths, img_size=(224, 224)):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, img_size)
        images.append(img)
    return np.array(images)


X_train = load_images(X_train)
X_val = load_images(X_val)
X_test = load_images(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:100]:  
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Aumentei o dropout para regularização
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)
model.save("detector.keras")
loss, acc = model.evaluate(X_test, y_test)

print(f'Test accuracy: {acc:.2f}')

In [7]:
"""
Use the model to detect dangerous objects in a video.
"""
from utils.emails import send_email


def detect_objects_in_video(video_path, model, threshold=0.5):
    cap = cv2.VideoCapture(video_path)
    flag = True

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        img = cv2.resize(frame, (224, 224)) / 255.0
        pred = model.predict(np.expand_dims(img, axis=0))[0][0]    
        
        label = 'DANGEROUS' if pred > threshold else 'SAFE'
        color = (0, 0, 255) if label == 'DANGEROUS' else (0, 255, 0)
        
        cv2.putText(frame, label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow('Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            send_email(frame)
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
detect_objects_in_video("videos/video.mp4", model)